# Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV, TimeSeriesSplit

# Importing the data

In [2]:
data_path = r"C:\Users\titou\Desktop\python_test_files\ML_classic\hackathon\store_sales_forecasting\data"

input_data_df = pd.read_csv(data_path + r'\train.csv')
#test_data_df = pd.read_csv(data_path + r'\test.csv')
holiday_df = pd.read_csv(data_path + r'\holidays_events.csv')
oil_df = pd.read_csv(data_path + r'\oil.csv')
stores_df = pd.read_csv(data_path + r'\stores.csv')
#transaction_df = pd.read_csv(data_path + r'\transactions.csv')

In [3]:
print("input data:", input_data_df.info(), "\n")
print("holiday:",holiday_df.info(), "\n")
print("oil:",oil_df.info(), "\n")
print("stores:",stores_df.info(), "\n")
#print("transaction:",transaction_df.info(), "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB
input data: None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
holiday: None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 col

In [4]:
input_data_extended_df = pd.merge

In [5]:
input_data_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [6]:
holiday_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [7]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [8]:
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


# Data Cleaning

### Cleaning holiday

In [9]:
holiday_df_clean = holiday_df[["date","type", "transferred"]].copy()
holiday_df_clean[["type", "transferred"]] = holiday_df[["type", "transferred"]].astype("category")
holiday_df_clean["type"] = holiday_df_clean["type"].cat.codes + 1
holiday_df_clean["transferred"] = holiday_df_clean["transferred"].cat.codes

In [10]:
holiday_df_clean = holiday_df_clean.rename(columns = {"type":"holiday_type","transferred": "holiday_transferred"})
holiday_df_clean

,date,holiday_type,holiday_transferred
0,2012-03-02,4,0
1,2012-04-01,4,0
2,2012-04-12,4,0
3,2012-04-14,4,0
4,2012-04-21,4,0
...,...,...,...
345,2017-12-22,1,0
346,2017-12-23,1,0
347,2017-12-24,1,0
348,2017-12-25,4,0


### Cleaning oil

In [69]:
oil_df_clean = oil_df.ffill().bfill()
oil_df_clean["dcoilwtico"].rolling(3, min_periods=1).mean()

0       93.140000
1       93.140000
2       93.083333
3       93.076667
4       93.096667
          ...    
1213    47.780000
1214    47.096667
1215    46.836667
1216    46.273333
1217    46.560000
Name: dcoilwtico, Length: 1218, dtype: float64

### Cleaning store

In [12]:
stores_df_clean = stores_df.copy()
stores_df_clean[["city", "state", "type", "cluster"]] = stores_df_clean[["city", "state", "type", "cluster"]].astype("category")
for col in ["city", "state", "type", "cluster"]:
    stores_df_clean[col] = stores_df_clean[col].cat.codes

# Data merging

In [26]:
print(input_data_df.isna().any(axis=0))
input_data_df.head()

id             False
date           False
store_nbr      False
family         False
sales          False
onpromotion    False
dtype: bool


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [27]:
print(holiday_df_clean.isna().any(axis=0))
holiday_df_clean.head()

date                   False
holiday_type           False
holiday_transferred    False
dtype: bool


,date,holiday_type,holiday_transferred
0,2012-03-02,4,0
1,2012-04-01,4,0
2,2012-04-12,4,0
3,2012-04-14,4,0
4,2012-04-21,4,0


In [79]:
print(oil_df_clean.isna().any(axis=0))
oil_df_clean.head()


LAGS = [1, 2, 3, 7, 14, 28]
for lag in LAGS:
    oil_df_clean[f"oil_lag_{lag}"] = oil_df_clean["dcoilwtico"].shift(lag)

oil_df_clean = oil_df_clean.bfill()

date          False
dcoilwtico    False
oil_lag_1      True
oil_lag_2      True
oil_lag_3      True
oil_lag_7      True
oil_lag_14     True
oil_lag_28     True
dtype: bool


In [80]:
oil_df_clean.isna().any(axis=0)

date          False
dcoilwtico    False
oil_lag_1     False
oil_lag_2     False
oil_lag_3     False
oil_lag_7     False
oil_lag_14    False
oil_lag_28    False
dtype: bool

In [29]:
print(stores_df_clean.isna().any(axis=0))
stores_df_clean.head()

store_nbr    False
city         False
state        False
type         False
cluster      False
dtype: bool


,store_nbr,city,state,type,cluster
0,1,18,12,3,12
1,2,18,12,3,12
2,3,18,12,3,7
3,4,18,12,3,8
4,5,21,14,3,3


In [30]:
input_data_merged_df = input_data_df.merge(holiday_df_clean, on="date", how="left")
input_data_merged_df = input_data_merged_df.fillna(0)
input_data_merged_df.isna().any(axis=0)

id                     False
date                   False
store_nbr              False
family                 False
sales                  False
onpromotion            False
holiday_type           False
holiday_transferred    False
dtype: bool

In [31]:
input_data_merged_df = input_data_merged_df.merge(oil_df_clean, on="date", how="left")
input_data_merged_df["dcoilwtico"] = input_data_merged_df["dcoilwtico"].ffill()
input_data_merged_df.head()

,id,date,store_nbr,family,sales,onpromotion,holiday_type,holiday_transferred,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,4.0,0.0,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0,4.0,0.0,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0,4.0,0.0,93.14
3,3,2013-01-01,1,BEVERAGES,0.0,0,4.0,0.0,93.14
4,4,2013-01-01,1,BOOKS,0.0,0,4.0,0.0,93.14


In [32]:
input_data_merged_df.isna().any(axis=0)

id                     False
date                   False
store_nbr              False
family                 False
sales                  False
onpromotion            False
holiday_type           False
holiday_transferred    False
dcoilwtico             False
dtype: bool

In [33]:
input_data_merged_df = input_data_merged_df.merge(stores_df_clean, on="store_nbr", how="left")
input_data_merged_df.isna().any(axis=0)

id                     False
date                   False
store_nbr              False
family                 False
sales                  False
onpromotion            False
holiday_type           False
holiday_transferred    False
dcoilwtico             False
city                   False
state                  False
type                   False
cluster                False
dtype: bool

In [34]:
input_data_merged_df["family"] = input_data_merged_df["family"].astype("category")
input_data_merged_df["family"] = input_data_merged_df["family"].cat.codes
input_data_merged_df = input_data_merged_df.drop(columns=[ "id"])

In [35]:
input_data_merged_df.describe()

,store_nbr,family,sales,onpromotion,holiday_type,holiday_transferred,dcoilwtico,city,state,type,cluster
count,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06
mean,2.750000e+01,1.600000e+01,3.590209e+02,2.617480e+00,5.665111e-01,5.250875e-03,6.785992e+01,1.255556e+01,8.796296e+00,2.000000e+00,7.481481e+00
std,1.558579e+01,9.521906e+00,1.107286e+03,1.225494e+01,1.354563e+00,7.227244e-02,2.566330e+01,6.264737e+00,4.165021e+00,1.201851e+00,4.649735e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.619000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.400000e+01,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.632000e+01,8.000000e+00,6.000000e+00,1.000000e+00,3.000000e+00
50%,2.750000e+01,1.600000e+01,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,5.341000e+01,1.400000e+01,1.000000e+01,2.000000e+00,7.500000e+00
75%,4.100000e+01,2.400000e+01,1.960110e+02,0.000000e+00,0.000000e+00,0.000000e+00,9.572000e+01,1.800000e+01,1.200000e+01,3.000000e+00,1.200000e+01
max,5.400000e+01,3.200000e+01,1.247170e+05,7.410000e+02,6.000000e+00,1.000000e+00,1.106200e+02,2.100000e+01,1.500000e+01,4.000000e+00,1.600000e+01


# Feature engineering

In [36]:
input_data_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   date                 object 
 1   store_nbr            int64  
 2   family               int8   
 3   sales                float64
 4   onpromotion          int64  
 5   holiday_type         float64
 6   holiday_transferred  float64
 7   dcoilwtico           float64
 8   city                 int8   
 9   state                int8   
 10  type                 int8   
 11  cluster              int8   
dtypes: float64(4), int64(2), int8(5), object(1)
memory usage: 177.7+ MB


In [ ]:
input_data_merged_df.head()

,date,store_nbr,family,sales,onpromotion,holiday_type,holiday_transferred,dcoilwtico,city,state,type,cluster
0,2013-01-01,1,0,0.0,0,4.0,0.0,93.14,18,12,3,12
1,2013-01-01,1,1,0.0,0,4.0,0.0,93.14,18,12,3,12
2,2013-01-01,1,2,0.0,0,4.0,0.0,93.14,18,12,3,12
3,2013-01-01,1,3,0.0,0,4.0,0.0,93.14,18,12,3,12
4,2013-01-01,1,4,0.0,0,4.0,0.0,93.14,18,12,3,12


### Timestamp data

In [55]:
input_data_merged_df["date"] = pd.to_datetime(input_data_merged_df["date"])
input_data_merged_df = input_data_merged_df.sort_values("date")

In [48]:
input_data_merged_df["day"] = input_data_merged_df["date"].dt.isocalendar().day
input_data_merged_df["week"] = input_data_merged_df["date"].dt.isocalendar().week
input_data_merged_df["year"] = input_data_merged_df["date"].dt.isocalendar().year

### Lags on non-groupby features

In [53]:
input_data_merged_df.tail()

,date,store_nbr,family,sales,onpromotion,holiday_type,holiday_transferred,dcoilwtico,city,state,type,cluster,day,week,year
3054343,2017-08-15,9,28,438.133,0,4.0,0.0,47.57,18,12,1,5,2,33,2017
3054344,2017-08-15,9,29,154.553,1,4.0,0.0,47.57,18,12,1,5,2,33,2017
3054345,2017-08-15,9,30,2419.729,148,4.0,0.0,47.57,18,12,1,5,2,33,2017
3054346,2017-08-15,9,31,121.000,8,4.0,0.0,47.57,18,12,1,5,2,33,2017
3054347,2017-08-15,9,32,16.000,0,4.0,0.0,47.57,18,12,1,5,2,33,2017


In [70]:
input_data_merged_df["sales_rolling_3"]=input_data_merged_df.groupby(["store_nbr","family"])["sales"].shift(1).rolling(3, min_periods=1).mean()
input_data_merged_df["sales_rolling_7"]=input_data_merged_df.groupby(["store_nbr","family"])["sales"].shift(1).rolling(7, min_periods=1).mean()
input_data_merged_df["sales_rolling_14"]=input_data_merged_df.groupby(["store_nbr","family"])["sales"].shift(1).rolling(14, min_periods=1).mean()


LAGS = [1, 2, 3, 7, 14, 28]
for lag in LAGS:
    input_data_merged_df[f"sales_lag_{lag}"] = (
        input_data_merged_df.groupby(["store_nbr", "family"])["sales"].shift(lag)
    )

In [82]:
input_data_merged_df.columns

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'holiday_type',
       'holiday_transferred', 'dcoilwtico', 'city', 'state', 'type', 'cluster',
       'day', 'week', 'year', 'sales_rolling_3', 'sales_rolling_7',
       'sales_rolling_14', 'sales_lag_1', 'sales_lag_2', 'sales_lag_3',
       'sales_lag_7', 'sales_lag_14', 'sales_lag_28'],
      dtype='object')

# Building the model

In [25]:
X = input_data_merged_df.drop(columns="sales").values
y = input_data_merged_df["sales"].values

In [ ]:
X.shape

(3054348, 10)

: 

In [ ]:
cv = TimeSeriesSplit(n_splits=5)
model = XGBRegressor()
scores = cross_validate(model, X, y, cv=cv, scoring=["neg_mean_squared_error", "r2"])
print(f"The scores are: {scores["test_score_r2"]}")

Exception ignored on calling ctypes callback function <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x0000024E15234640>>:
Traceback (most recent call last):
  File "c:\Users\titou\Desktop\python_test_files\virtual_environments\ml_env\Lib\site-packages\xgboost\core.py", line 630, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument
KeyboardInterrupt: 


In [ ]:
print(f"The scores are: {scores}")

The scores are: {'fit_time': array([0.61606741, 1.15825558, 1.89236164, 2.66828179, 3.57439661]), 'score_time': array([0.12247491, 0.1213398 , 0.12253857, 0.12633324, 0.13281083]), 'test_neg_mean_squared_error': array([-231438.30351633, -298645.56747345, -488347.86409576,
       -334444.96454354, -263944.39743065]), 'test_r2': array([0.70814   , 0.71113397, 0.66153567, 0.79539618, 0.86174464])}
